# Installing GATK + notes
    This page will be a reference for installation, different tutorials and how to call polyploids

  #### Get GATK jar and .yml files here:
  https://github.com/broadinstitute/gatk/releases
  
  wget .zip file to a src directory, unzip, run conda install below in that direcoty
  
  #### directions for conda install here:
  https://gatk.broadinstitute.org/hc/en-us/articles/360035889851--How-to-Install-and-use-Conda-for-GATK4
  
  #### vivaswat notes
  https://microcollaborative.atlassian.net/wiki/spaces/BL/pages/73957377/Bioinformatics+for+polyploids
  
  #### monnahan et al 2019   
  
  https://github.com/pmonnahan/ArenosaPloidy

  
  
  #### IDEA: 
  
  - call snps with GATK haplotype caller. 
  - run one per sample with correct ploidy into gvcf
  - then use GenotypeGVCF to call snps across all gvcfs, no need to tell ploidy. 
  
  **combine gvcf**
-R -o -V

**preferred: make genome DB**
gatk GenomicsDBImport \
    -V data/gvcfs/mother.g.vcf \
    -V data/gvcfs/father.g.vcf \
    -V data/gvcfs/son.g.vcf \
    --genomicsdb-workspace-path my_database \

**genotype**
-R -o -L --includeNonVariantSites -nt 5

**select biallelic**
-R -o -V -xlSelectType INDEL -xlSelectType MIXED --restrictAllelesTo BIALLELIC

**filter**

  

In [1]:
import sys

#sys.path.append('/home/faske/g/anaconda3/envs/py34/lib/python3.4/site-packages')
sys.path.append('/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/data/gpfs/assoc/parchmanlab/tfaske/ipynb/include_utils")

import ipyparallel as ipp
import os, time
from os import environ
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
import re
from itertools import chain
import socket
import random
#import Levenshtein as lv

In [2]:
root = "/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo"

In [3]:
snp_dir = os.path.join(root,'SNPcall')

In [4]:
gatk_dir = os.path.join(snp_dir,'gatk')

In [5]:
bam_dir = os.path.join(gatk_dir,'good_bams')

In [6]:
cd $bam_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/good_bams


In [7]:
gatk = '/data/gpfs/home/tfaske/g/src/gatk-4.2.3.0/gatk'

In [15]:
#assembly = os.path.join(gatk_dir, "mapped_fasta/mapped.fasta")
assembly = "/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/assembly/reference.fasta"
assert os.path.exists(assembly)
assembly
#picard CreateSequenceDictionary R=reference.fasta O=reference.dict

'/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/assembly/reference.fasta'

In [148]:
mapped_bam = []
files = !find . -type f -name '*mapped.bam'
files = [os.path.abspath(x) for x in files if 'bam' in x]
for x in files:
    mapped_bam.append(x)
mapped_bam = sorted(mapped_bam)

In [149]:
len(mapped_bam), mapped_bam[0]

(695,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/good_bams/AT2_BB_10_mapped.bam')

## SNP CALL with GATK

follow that of mohannon et al 2019 nature ecology and evolution

-I -R -o --min_base_quality_score 25 --min_mapping_quality_score 25 -rf DuplicateRead -rf BadMate -rf BadCigar -ERC BP_RESOLUTION -R -L -o <fileName_g.vcf.gz> -ploidy -stand_emit_conf 25 -stand_call_conf 25 --pcr_indel_model NONE -nct 4!

In [151]:
cd $gatk_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk


In [152]:
!mkdir vcf

mkdir: cannot create directory ‘vcf’: File exists


In [154]:
def run_gatk(args):
    import os, multiprocessing, socket
    gatk, assembly, bam = args
    ID = bam.split('/')[11] ### need to change this to match your ID 
    ID = ID.split('_mapped.bam')[0] ### This too
    ploidy = ID.split('_')[0]
    ploidy = re.sub(r'\D','',ploidy)
    #print(ID)
    #print(ploidy)
    #print(out_dir)
    cmd = r'%s --java-options "-Xmx16g" HaplotypeCaller -R %s -I %s -ploidy %s -O vcf/%s.g.vcf.gz -ERC BP_RESOLUTION' % (gatk,assembly,bam,ploidy,ID)
    #cmd = r'%s --java-options "-Xmx16g" HaplotypeCaller -R %s -I %s -ploidy %s -O %s/%s.g.vcf.gz -ERC GVCF' % (gatk,assembly,bam,ploidy,out_dir,ID)
    return  cmd 

In [155]:
### creates a list of commands for gatk for each bam
res_gatk = []
for b in mapped_bam:
    res = run_gatk((gatk, assembly, b))
    res_gatk.append(res)

In [156]:
len(res_gatk),res_gatk[0]

(695,
 '/data/gpfs/assoc/parchmanlab/tfaske/src/gatk4/gatk --java-options "-Xmx16g" HaplotypeCaller -R /data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/assembly/reference.fasta -I /data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/good_bams/AT2_BB_10_mapped.bam -ploidy 2 -O vcf/AT2_BB_10.g.vcf.gz -ERC BP_RESOLUTION')

#### Below selects options for slurm submission and is a function for creating a slurm script per fastq

In [157]:
cd $gatk_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk


In [158]:
!mkdir shdir

mkdir: cannot create directory ‘shdir’: File exists


In [159]:
bam_ID = [b.split('/')[11].split('_mapped.bam')[0] for b in mapped_bam]

In [160]:
len(bam_ID), bam_ID[0]

(695, 'AT2_BB_10')

In [161]:
### select options for slurm submission
#account = 'cpu-s5-denovo-0'
#partition = 'cpu-core-0'
account = 'cpu-s1-bionres-0'
partition = 'cpu-s1-bionres-0'
jobname = 'gatk'
time = '1-00:00:00' #time limit 1 day
cpus = 1
mem_cpu = 8000
email = 'tfaske@nevada.unr.edu'

In [162]:
def write_gatk_ind_sh(account,partition,time,jobname,cpus,mem_cpu,email,bam_ID,gatk_dir,cmds):
    #print(account)
    #print(partition)
    #print(cpus)
    #print(email)
    #print(cmds[0])
    for i in range(0,len(cmds)):
        with open("shdir/run_gatk_%s.sh" % (bam_ID[i]), "w") as o:
            o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name %s_gatk
#SBATCH --output shdir/output_gatk_%s.txt
#SBATCH --mail-type=FAIL
#SBATCH --mail-user=%s \n

cd %s\n
    
%s \n
""" % (account,partition,time,int(cpus),int(mem_cpu),bam_ID[i],bam_ID[i],email,gatk_dir,cmds[i]))
            

In [163]:
write_gatk_ind_sh(account,partition,time,jobname,cpus,mem_cpu,email,bam_ID,gatk_dir,res_gatk)

#### finds all bwa slurm scripts and writes bash script to sbatch them

In [164]:
shbwa_files = []
os.chdir('{}/{}'.format(gatk_dir, 'shdir'))
files = !find . -name '*.sh'
files = [os.path.abspath(x) for x in files]
for x in files:
        shbwa_files.append(x)
shbwa_files = sorted(shbwa_files)

In [165]:
len(shbwa_files),shbwa_files[0]

(695,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/shdir/run_gatk_AT2_BB_1.sh')

In [166]:
def write_bash_gatk_sh(sh_files):
    with open("run_bash_gatk.sh", "w") as o:
        o.write("""#!/usr/bin/env bash \n\n""")
        for f in sh_files:
            o.write("sbatch %s \n" % (f))    

In [167]:
cd $gatk_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk


In [168]:
write_bash_gatk_sh(shbwa_files)

### Run run_bash_gatk_sh locally
    cd /data/gpfs/assoc/denovo/tfaske/sagebrush/SNPcall 
    source activate py36
    bash run_bash_gatk.sh


## Reblock gvcf

https://gatk.broadinstitute.org/hc/en-us/articles/4405443600667-ReblockGVCF

This strategy drops Uncalled alleles and associated data will also be dropped unless --keep-all-alts is specified.

This will help with GenomicsDBimport.

 gatk ReblockGVCF \
   -R reference.fasta \
   -V sample1.g.vcf \
   -O sample1.rb.g.vcf
   -ploidy 

In [393]:
cd $gatk_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk


In [364]:
!mkdir reblock

In [365]:
!mkdir reblock/shdir

In [394]:
vcfs = !ls vcf/*g.vcf.gz | grep -v tbi | grep -v sorted
vcfs = [os.path.abspath(x) for x in vcfs]

In [395]:
len(vcfs),vcfs[0]

(695,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/vcf/AT2_BB_10.g.vcf.gz')

In [418]:
def run_rb(args):
    gatk, assembly, vcf = args
    ID = vcf.split('/')[11] ### need to change this to match your ID 
    ID = ID.split('.g.vcf.gz')[0] ### This too
    ploidy = ID.split('_')[0]
    ploidy = re.sub(r'\D','',ploidy)
    cmd = r'%s --java-options "-Xmx2g" ReblockGVCF -R %s -V %s -ploidy %s -O reblock/%s.rb.g.vcf.gz' % (gatk,assembly,vcf,ploidy,ID)
    return  cmd 

In [419]:
rb_cmds = []
for vcf in vcfs: 
    rb_cmds.append(run_rb((gatk, assembly, vcf)))

In [420]:
len(rb_cmds),rb_cmds[250]

(695,
 '/data/gpfs/assoc/parchmanlab/tfaske/src/gatk4/gatk --java-options "-Xmx2g" ReblockGVCF -R /data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/assembly/reference.fasta -V /data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/vcf/AT4_WT_2.g.vcf.gz -ploidy 4 -O reblock/AT4_WT_2.rb.g.vcf.gz')

In [421]:
### select options for slurm submission
#account = 'cpu-s5-denovo-0'
#partition = 'cpu-core-0'
account = 'cpu-s1-bionres-0'
partition = 'cpu-s1-bionres-0'
jobname = 'gatk'
time = '0-01:00:00' #time limit 1 hour
cpus = 1
mem_cpu = 2000
email = 'tfaske@nevada.unr.edu'

In [422]:
def write_rb_sh(account,partition,time,jobname,cpus,mem_cpu,email,gatk_dir,cmds,vcfs):
    for i in range(0,len(cmds)):
        ID = vcfs[i].split('/')[11].split('.g.vcf.gz')[0]
        with open("reblock/shdir/run_rb_%s.sh" % (ID), "w") as o:
            o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name rb_%s
#SBATCH --output reblock/shdir/output_rb_%s.txt
#SBATCH --mail-type=FAIL
#SBATCH --mail-user=%s \n

cd %s\n
    
%s\n""" % (account,partition,time,int(cpus),int(mem_cpu),ID,ID,email,gatk_dir,cmds[i]))
            

In [423]:
write_rb_sh(account,partition,time,jobname,cpus,mem_cpu,email,gatk_dir,rb_cmds,vcfs)

**finds all rb slurm scripts and writes bash script to sbatch them**

In [413]:
shrb_files = []
os.chdir('{}/{}'.format(gatk_dir, 'reblock/shdir'))
files = !find . -name '*.sh'
files = [os.path.abspath(x) for x in files]
for x in files:
        shrb_files.append(x)
shrb_files = sorted(shrb_files)

In [414]:
len(shrb_files),shrb_files[0]

(695,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/reblock/shdir/run_rb_AT2_BB_1.sh')

In [415]:
def write_bash_rb_sh(sh_files):
    with open("run_bash_rb.sh", "w") as o:
        o.write("""#!/usr/bin/env bash \n\n""")
        for f in sh_files:
            o.write("sbatch %s \n" % (f))  

In [416]:
cd $gatk_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk


In [417]:
write_bash_rb_sh(shrb_files)

## consider CombineGVCFs instead

GenomicsDBimport is better but a nightmare

https://gatk.broadinstitute.org/hc/en-us/articles/360037053272-CombineGVCFs

 gatk CombineGVCFs \
   -R reference.fasta \
   --variant sample1.g.vcf.gz \
   --variant sample2.g.vcf.gz \
   -O cohort.g.vcf.gz

In [ ]:
cd $gatk_dir

In [424]:
!mkdir cohort

In [435]:
vcfs = !ls reblock/*rb.g.vcf.gz | grep -v tbi | grep -v sorted
vcfs = [os.path.abspath(x) for x in vcfs]

In [436]:
len(vcfs),vcfs[0]

(695,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/reblock/AT2_BB_10.rb.g.vcf.gz')

In [445]:
with open("cohort/vcf.list", "w") as o:
    for vcf in vcfs:
        o.write("%s\n" % (vcf))

In [454]:
### select options for slurm submission
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
#account = 'cpu-s1-bionres-0'
#partition = 'cpu-s1-bionres-0'
jobname = 'gatk'
time = '2-00:00:00' #time limit 1 day
cpus = 1
mem_cpu = 80000
email = 'tfaske@nevada.unr.edu'

In [460]:
def write_combine_sh(account,partition,time,jobname,cpus,mem_cpu,email,bam_ID,gatk,gatk_dir,assembly):
    with open("run_combineGVCF.sh", "w") as o:
        o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name combineGVCF
#SBATCH --output cohort/output_combineGVCF.txt
#SBATCH --mail-type=FAIL
#SBATCH --mail-user=%s \n

cd %s\n
    
%s --java-options "-Xmx80g" CombineGVCFs -R %s -V cohort/vcf.list -O cohort/cohort.rb.g.vcf.gz \n
""" % (account,partition,time,int(cpus),int(mem_cpu),email,gatk_dir,gatk,assembly))
            

In [461]:
write_combine_sh(account,partition,time,jobname,cpus,mem_cpu,email,bam_ID,gatk,gatk_dir,assembly)

### GenomicsDBimport 

    gatk --java-options "-Xmx4g -Xms4g" \
       GenomicsDBImport \
       --genomicsdb-workspace-path my_database \
       --batch-size 50 \
       --sample-name-map cohort.sample_map \
       --tmp-dir=/path/to/large/tmp \
       --reader-threads 2
       -L interval.list
       
   **sample map**  
     sample1      sample1.vcf.gz  
     sample2      sample2.vcf.gz  
     sample3      sample3.vcf.gz  
     
   **interval list**
   -L intervals.list  
   *chr* \n  
    
   **this step takes forever.**  
   make 100 interval lists to submit 100 jobs. 
   
   **check this out for file issues**   
   https://gatk.broadinstitute.org/hc/en-us/articles/360056138571-GenomicsDBImport-usage-and-performance-guidelines
   
    As of this writing, an upcoming GATK release will include a feature that will substantially boost the niche case where users have many small contigs (>100, although there are cases with over 500k). In this case, the --merge-input-intervals flag will not help, and so we suggest using the --merge-contigs-into-num-partitions and setting it to something less than 100.
    
    --bypass-feature-reader 
    
      Use the consolidate flag if more than a hundred batches were used. This will improve
                              feature read time.

In [11]:
cd $gatk_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk


In [42]:
!mkdir db

In [43]:
!mkdir db/shdir

In [44]:
!mkdir intervals

mkdir: cannot create directory ‘intervals’: File exists


**Use reblock g.vcf**

In [12]:
vcfs = !ls reblock/*rb.g.vcf.gz | grep -v tbi | grep -v sorted
vcfs = [os.path.abspath(x) for x in vcfs]

In [13]:
len(vcfs),vcfs[0]

(695,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/reblock/AT2_BB_10.rb.g.vcf.gz')

**get pos file to make interval list**

zgrep "^[^#]" file.vcf.gz | awk '{print $2}'

In [29]:
f = vcfs[0]
environ['f'] = f
interval = !zgrep "^[^#]" $f | awk '{print $1}'

In [30]:
interval_list = [x for x in interval]

In [31]:
interval_list = list(set(interval_list))
len(interval_list),interval_list[:10]

(61970,
 ['dDocent_Contig_124214',
  'dDocent_Contig_113311',
  'dDocent_Contig_128784',
  'dDocent_Contig_187280',
  'dDocent_Contig_3797',
  'dDocent_Contig_245058',
  'dDocent_Contig_45492',
  'dDocent_Contig_356422',
  'dDocent_Contig_229452',
  'dDocent_Contig_264425'])

In [32]:
with open("intervals/interval_all.txt", "w") as o:
    for i in interval_list:
        o.write("%s\n" % (i))

In [33]:
def create_split_intervals(nodes, interval):
    lines = 0
    for line in open(interval):
        lines += 1
    print(lines, lines//nodes)
    per_interval = lines//nodes
    cmd = "split -a 3 -d -l %d %s intervals/interval." % (per_interval, interval)
    !$cmd
    #call(cmd.split())
create_split_intervals(99, "intervals/interval_all.txt") #nodes -1 

61970 625


In [34]:
intervals = !ls intervals/interval.*
intervals = [os.path.abspath(x) for x in intervals]

In [35]:
len(intervals),intervals[0]

(100,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/intervals/interval.000')

In [36]:
## rename intervals to end in .list
for i in intervals:
    new_int = i + '.list'
    !mv $i $new_int

In [37]:
def make_sample_maps(vcfs):
    with open("reblock/sample_map.txt", "w") as o:
        for vcf in vcfs: 
            sample = vcf.split('/')[11].split('.rg.g.vcf.gz')[0]
            #print(sample)
            o.write("%s\t%s\n" % (sample,vcf))

In [38]:
make_sample_maps(vcfs)

In [54]:
def run_gen_db(args):
    gatk,inter = args
    cmd = r'%s --java-options "-Xmx16g -Xms16g" GenomicsDBImport --genomicsdb-workspace-path db/gen_db.%s --batch-size 50 --sample-name-map vcf/sample_map.txt -L intervals/interval.%s.list --tmp-dir db/tmp.%s --consolidate true --bypass-feature-reader --merge-contigs-into-num-partitions 50' % (gatk,inter,inter,inter)
    return  cmd 

In [55]:
db_cmds = []
for inter in intervals: 
    i = inter.split('.')[1]
    #print(i)
    db_cmds.append(run_gen_db((gatk,i)))

In [56]:
len(db_cmds),db_cmds[0]

(100,
 '/data/gpfs/home/tfaske/g/src/gatk-4.2.3.0/gatk --java-options "-Xmx16g -Xms16g" GenomicsDBImport --genomicsdb-workspace-path db/gen_db.000 --batch-size 50 --sample-name-map vcf/sample_map.txt -L intervals/interval.000.list --tmp-dir db/tmp.000 --consolidate true --bypass-feature-reader --merge-contigs-into-num-partitions 50')

**make db/tmp.XXX dir**

In [57]:
for inter in intervals: 
    i = inter.split('.')[1]
    tmp_dir = 'db/tmp.' + i
    !mkdir $tmp_dir

mkdir: cannot create directory ‘db/tmp.000’: File exists
mkdir: cannot create directory ‘db/tmp.001’: File exists
mkdir: cannot create directory ‘db/tmp.002’: File exists
mkdir: cannot create directory ‘db/tmp.003’: File exists
mkdir: cannot create directory ‘db/tmp.004’: File exists
mkdir: cannot create directory ‘db/tmp.005’: File exists
mkdir: cannot create directory ‘db/tmp.006’: File exists
mkdir: cannot create directory ‘db/tmp.007’: File exists
mkdir: cannot create directory ‘db/tmp.008’: File exists
mkdir: cannot create directory ‘db/tmp.009’: File exists
mkdir: cannot create directory ‘db/tmp.010’: File exists
mkdir: cannot create directory ‘db/tmp.011’: File exists
mkdir: cannot create directory ‘db/tmp.012’: File exists
mkdir: cannot create directory ‘db/tmp.013’: File exists
mkdir: cannot create directory ‘db/tmp.014’: File exists
mkdir: cannot create directory ‘db/tmp.015’: File exists
mkdir: cannot create directory ‘db/tmp.016’: File exists
mkdir: cannot create directory 

UnboundLocalError: local variable 'child' referenced before assignment

In [58]:
### select options for slurm submission
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
#account = 'cpu-s1-bionres-0'
#partition = 'cpu-s1-bionres-0'
jobname = 'gatk'
time = '2-00:00:00' #time limit 1 day
cpus = 1
mem_cpu = 16000
email = 'tfaske@nevada.unr.edu'

In [59]:
def write_GenomicsDB_sh(account,partition,time,jobname,cpus,mem_cpu,email,gatk_dir,cmds,intervals):
    for i in range(0,len(cmds)):
        inter = intervals[i].split('.')[1]
        with open("db/shdir/run_db_%s.sh" % (inter), "w") as o:
            o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name db_%s
#SBATCH --output db/shdir/output_db_%s.txt
#SBATCH --mail-type=FAIL
#SBATCH --mail-user=%s \n

cd %s\n
    
%s\n""" % (account,partition,time,int(cpus),int(mem_cpu),inter,inter,email,gatk_dir,cmds[i]))
            

In [60]:
write_GenomicsDB_sh(account,partition,time,jobname,cpus,mem_cpu,email,gatk_dir,db_cmds,intervals)

**finds all db slurm scripts and writes bash script to sbatch them**

In [61]:
shdb_files = []
os.chdir('{}/{}'.format(gatk_dir, 'db/shdir'))
files = !find . -name '*.sh'
files = [os.path.abspath(x) for x in files]
for x in files:
        shdb_files.append(x)
shdb_files = sorted(shdb_files)

In [62]:
len(shdb_files),shdb_files[0]

(100,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/db/shdir/run_db_000.sh')

In [63]:
def write_bash_db_sh(sh_files):
    with open("run_bash_db.sh", "w") as o:
        o.write("""#!/usr/bin/env bash \n\n""")
        for f in sh_files:
            o.write("sbatch %s \n" % (f))  

In [64]:
cd $gatk_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk


In [65]:
write_bash_db_sh(shdb_files)

#### Run run_bash_fb_sh locally
    cd /data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk
    source activate py36
    bash run_bash_db.sh

### Genotype 

https://gatk.broadinstitute.org/hc/en-us/articles/360037057852-GenotypeGVCFs

 gatk --java-options "-Xmx16g" GenotypeGVCFs \
   -R Homo_sapiens_assembly38.fasta \
   -V gendb://my_database \
   -O output.vcf.gz
   
   **need to genotype each interval** 
   
   --max-genotype-count limit is 1024. Consider changing to higher since 695*4 individuals. Got an error for a couple loci: too many genotypes in the combined VCF record = 3060
   
   **bug in that version of gatk: use 4.2.6**

In [8]:
cd $gatk_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk


In [47]:
gatk = '/data/gpfs/assoc/parchmanlab/tfaske/src/gatk-4.2.6.1/gatk'

In [9]:
!mkdir genotype

mkdir: cannot create directory ‘genotype’: File exists


In [10]:
!mkdir genotype/shdir

mkdir: cannot create directory ‘genotype/shdir’: File exists


In [11]:
#find gen_db
gen_db_files = !find ./db -type d -name gen_db.*
gen_db_files = [os.path.abspath(x) for x in gen_db_files]

In [12]:
len(gen_db_files),gen_db_files[0]

(100,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/db/gen_db.099')

In [70]:
def run_gt(args):
    gatk,assembly,db,inter = args
    cmd = r'%s --java-options "-Xmx16g" GenotypeGVCFs -R %s -V gendb://%s --max-genotype-count 8196 -O genotype/gatk.%s.vcf.gz' % (gatk,assembly,db,inter)
    return  cmd 

In [71]:
gt_cmds = []
for db in gen_db_files: 
    inter = db.split('.')[1]
    #print(i)
    gt_cmds.append(run_gt((gatk,assembly,db,inter)))

In [72]:
len(gt_cmds),gt_cmds[1]

(100,
 '/data/gpfs/assoc/parchmanlab/tfaske/src/gatk-4.2.6.1/gatk --java-options "-Xmx16g" GenotypeGVCFs -R /data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/assembly/reference.fasta -V gendb:///data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/db/gen_db.091 --max-genotype-count 8196 -O genotype/gatk.091.vcf.gz')

In [73]:
### select options for slurm submission
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
#account = 'cpu-s1-bionres-0'
#partition = 'cpu-s1-bionres-0'
jobname = 'gatk'
time = '1-00:00:00' #time limit 1 day
cpus = 1
mem_cpu = 16000
email = 'tfaske@nevada.unr.edu'

In [74]:
def write_gt_sh(account,partition,time,jobname,cpus,mem_cpu,email,gatk_dir,cmds,files):
    for i in range(0,len(cmds)):
        inter = files[i].split('.')[1]
        with open("genotype/shdir/run_gt_%s.sh" % (inter), "w") as o:
            o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name gt_%s
#SBATCH --output genotype/shdir/output_gt_%s.txt
#SBATCH --mail-type=FAIL
#SBATCH --mail-user=%s \n

cd %s\n
    
%s\n""" % (account,partition,time,int(cpus),int(mem_cpu),inter,inter,email,gatk_dir,cmds[i]))
            

In [75]:
write_gt_sh(account,partition,time,jobname,cpus,mem_cpu,email,gatk_dir,gt_cmds,gen_db_files)

**finds all db slurm scripts and writes bash script to sbatch them**

In [76]:
shgt_files = []
os.chdir('{}/{}'.format(gatk_dir, 'genotype/shdir'))
files = !find . -name '*.sh'
files = [os.path.abspath(x) for x in files]
for x in files:
        shgt_files.append(x)
shgt_files = sorted(shgt_files)

In [77]:
len(shgt_files),shgt_files[0]

(100,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/genotype/shdir/run_gt_000.sh')

In [78]:
def write_bash_gt_sh(sh_files):
    with open("run_bash_gt.sh", "w") as o:
        o.write("""#!/usr/bin/env bash \n\n""")
        for f in sh_files:
            o.write("sbatch %s \n" % (f))  

In [79]:
cd $gatk_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk


In [80]:
write_bash_gt_sh(shgt_files)

#### Run run_bash_gt_sh locally
    cd /data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk
    source activate py36
    bash run_bash_gt.sh

### Keep only biallelic 

https://gatk.broadinstitute.org/hc/en-us/articles/360037055952-SelectVariants

SelectVariants -R -o -V --select-type-to-include SNP --restrict-alleles-to BIALLELIC

In [113]:
cd $gatk_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk


In [114]:
vcfs = !ls genotype/*vcf.gz | grep -v tbi | grep -v sorted
vcfs = [os.path.abspath(x) for x in vcfs]

In [115]:
len(vcfs),vcfs[0]

(100,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/genotype/gatk.000.vcf.gz')

In [134]:
def run_sv(args):
    gatk,assembly,vcf,inter = args
    cmd = r'%s --java-options "-Xmx16g" SelectVariants -R %s -V %s --select-type-to-include SNP --restrict-alleles-to BIALLELIC -O genotype/gatk.%s.biallelic.vcf.gz' % (gatk,assembly,vcf,inter)
    return  cmd 

In [135]:
sv_cmds = []
for vcf in vcfs: 
    inter = vcf.split('.')[1]
    #print(inter)
    sv_cmds.append(run_sv((gatk,assembly,vcf,inter)))

In [136]:
len(sv_cmds),sv_cmds[1]

(100,
 '/data/gpfs/assoc/parchmanlab/tfaske/src/gatk-4.2.6.1/gatk --java-options "-Xmx16g" SelectVariants -R /data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/assembly/reference.fasta -V /data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/genotype/gatk.001.vcf.gz --select-type-to-include SNP --restrict-alleles-to BIALLELIC -O genotype/gatk.001.biallelic.vcf.gz')

In [137]:
### select options for slurm submission
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
#account = 'cpu-s1-bionres-0'
#partition = 'cpu-s1-bionres-0'
jobname = 'gatk'
time = '1-00:00:00' #time limit 1 day
cpus = 1
mem_cpu = 16000
email = 'tfaske@nevada.unr.edu'

In [138]:
def write_sv_sh(account,partition,time,jobname,cpus,mem_cpu,email,gatk_dir,cmds,files):
    for i in range(0,len(cmds)):
        inter = files[i].split('.')[1]
        with open("genotype/shdir/run_sv_%s.sh" % (inter), "w") as o:
            o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name sv_%s
#SBATCH --output genotype/shdir/output_sv_%s.txt
#SBATCH --mail-type=FAIL
#SBATCH --mail-user=%s \n

cd %s\n
    
%s\n""" % (account,partition,time,int(cpus),int(mem_cpu),inter,inter,email,gatk_dir,cmds[i]))
            

In [139]:
write_sv_sh(account,partition,time,jobname,cpus,mem_cpu,email,gatk_dir,sv_cmds,vcfs)

**finds all sv slurm scripts and writes bash script to sbatch them**

In [140]:
shsv_files = !ls genotype/shdir/run_sv*sh
shsv_files = [os.path.abspath(x) for x in shsv_files]

In [141]:
len(shsv_files),shsv_files[0]

(100,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/genotype/shdir/run_sv_000.sh')

In [142]:
def write_bash_sv_sh(sh_files):
    with open("run_bash_sv.sh", "w") as o:
        o.write("""#!/usr/bin/env bash \n\n""")
        for f in sh_files:
            o.write("sbatch %s \n" % (f))  

In [143]:
cd $gatk_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk


In [144]:
write_bash_sv_sh(shsv_files)

### concat vcf files 

bcftools concat --threads 50 -Oz -o

In [146]:
cd $gatk_dir

/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk


In [150]:
bi_vcfs = !ls genotype/*biallelic.vcf.gz | grep -v tbi | grep -v sorted
bi_vcfs = [os.path.abspath(x) for x in bi_vcfs]

In [151]:
len(bi_vcfs),bi_vcfs[0]

(100,
 '/data/gpfs/assoc/denovo/tfaske/sagebrush/denovo/SNPcall/gatk/genotype/gatk.000.biallelic.vcf.gz')

In [164]:
concat_cmd = "bcftools concat --threads 2 -Oz -o genotype/gatk.ALL.vcf.gz %s "% (" ".join(bi_vcfs))

In [165]:
#concat_cmd

In [166]:
### select options for slurm submission
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
#account = 'cpu-s1-bionres-0'
#partition = 'cpu-s1-bionres-0'
jobname = 'gatk'
time = '1-00:00:00' #time limit 1 day
cpus = 1
mem_cpu = 80000
email = 'tfaske@nevada.unr.edu'

In [167]:
def write_concat_sh(account,partition,time,jobname,cpus,mem_cpu,email,gatk_dir,cmd):
    with open("run_concat.sh", "w") as o:
        o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name concat
#SBATCH --output genotype/output_concat.txt
#SBATCH --mail-type=FAIL
#SBATCH --mail-user=%s \n

cd %s\n
    
%s\n""" % (account,partition,time,int(cpus),int(mem_cpu),email,gatk_dir,cmd))
            

In [168]:
write_concat_sh(account,partition,time,jobname,cpus,mem_cpu,email,gatk_dir,concat_cmd)

In [171]:
!zcat genotype/gatk.ALL.vcf.gz | grep -v '#' | wc -l 

1303563


### begin filtering! 